Digital Signal Processing Courseware: An Introduction (copyright © 2024)
## Authors: J. Christopher Edgar and Gregory A. Miller

Originally written in Mathematica by J. Christopher Edgar. Conversion to Jupyter Notebook by Song Liu.

The authors of this courseware are indebted to Prof. Bruce Carpenter (University of Illinois Urbana-Champaign). Bruce inspired the creation of this courseware, he consulted with the authors as this courseware was being developed, and he provided the original version of the code and text for several sections of this courseware (e.g. the section on complex numbers and the section on normal distributions). 

# <font color=red>DSP.03 Convolution and Filtering - Time Domain</font>

# <font color=red>Give it a Try!</font>
# <font color=red>Part 3</font>

### Setup

In [1]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image as img
from matplotlib import cm
from mpl_toolkits import mplot3d
from scipy.fft import fft, fftfreq
import matplotlib.patches as patches
import math
import cmath
import pandas as pd
from sympy import Symbol, sin, series
from sympy import roots, solve_poly_system
import scipy.special

import warnings
warnings.filterwarnings('ignore')

# Figure size 
plt.rc("figure", figsize=(8, 6))

#function to create time course figure
#one waveform
def make_plot_1(x1,y1,type="b",linewidth = 1): 
    plt.plot(x1, y1,type)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#two overlaid waveforms with red and blue   
def make_plot_2(x1,y1,type1,x2,y2,type2): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#three overlaid waveforms with red, blue and green   
def make_plot_3(x1,y1,type1,x2,y2,type2,x3,y3,type3): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.plot(x3, y3, type3)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
def make_plot_3d(ax,x,y,z):    
    ax.contour3D(x, y, z, 50, cmap=cm.coolwarm)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
def make_plot_freq_1(x1,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf = fft(x1)
    xf = fftfreq(N, 1 / sample_rate)
    yf = yf[0:Nhalf]
    xf = xf[0:Nhalf]
    plt.plot(xf, np.abs(yf))
    
#two spectrums
def make_plot_freq_2(x1,x2,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf1 = fft(x1)
    yf2 = fft(x2)
    xf = fftfreq(N, 1 / sample_rate)

    yf1 = yf1[0:Nhalf]
    yf2 = yf2[0:Nhalf]
    xf = xf[0:Nhalf]

    plt.plot(xf, np.abs(yf1))
    plt.plot(xf, np.abs(yf2), color = 'r')
    
def make_imshow(x):
    plt.imshow(x,cmap='Greys_r')
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def make_imshow_color(x):
    plt.imshow(x)
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def round_complex(x):
    return complex(np.round(x.real,4),np.round(x.imag,4))

## <font color=red>DSP.03.G3) More on Global Warming</font>

### <font color=red>DSP.03.G3.a) Global temperatures from CE 200 to CE 1980</font>

Mann, M.E. and P.D. Jones, 2003, 2,000 Year Hemispheric Multi-proxy Temperature Reconstructions, IGBP PAGES/World Data Center for Paleoclimatology, Data Contribution Series #2003-051. NOAA/NGDC Paleoclimatology Program, Boulder CO, USA. ftp://ftp.ncdc.noaa.gov/pub/data/paleo/contributions_by_author/mann2003b/mann2003b.txt

Is the global temperature increasing?

In the Basics we looked as global temperature from 1880 to 2005. Let's now look at data from a study
published by Michael Mann and Philip D. Jones (2003). They noted that, although studies examining
temperature changes over the last several hundred years are of interest, findings from these studies are
limited, because the changes we have recently observed may be normal temperature fluctuations when
considered against a longer timeline. Mann and
Jones (2003) used a variety of methods to reconstruct Northern Hemisphere mean surface temperature
over the past two millennia. Temperature measurements from long ago are based on what Mann
and Jones called high-resolution 'proxy' temperature data. 'Proxy' temperature measures are obtained
by estimating temperatures across the last 2000 years from samples obtained from lake sediment, ice
core, and tree-ring records. For example, tree-ring temperature reconstructions were obtained for
western North American regions from ultra-long-lived trees from CE 200 and ice core oxygen isotope
records from west Greenland that date back to CE 553. If you are interested in the specific methods and
a full discussion of their findings, check out the paper. For present purposes, we just care that it's a very long timeseries.

Load and plot the needed data.

Use the below command, loading in the file 'JonesandMann2003.txt':

In [2]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

#find and select file "JonesandMann2003.txt"
file_path = filedialog.askopenfilename()
JonesandMann = pd.read_csv(file_path,delimiter = "\t")
JonesandMann = JonesandMann.to_numpy().flatten()
JonesandMann

array([-0.235, -0.217, -0.2  , ...,  0.027,  0.054,  0.08 ])

This is a list of the year a temperature measurement was obtained, followed by the temperature measurements. There are 1781 pairs of data points: all the years, then the temperature for those years.


For analyses, only the temperature data are needed. The year information can be removed, leaving only the temperature
measures.

Plot the temperature data and set the x axis so it is correct.

### <font color=red>DSP.03.G3.b) Global temperatures from CE 200 to CE 1980 - a better looking plot</font>

Mann and Jones (2003) noted that the proxy data are best interpreted in terms of annual conditions.
Mann and Jones (2003) noted that the broad period from approximately CE 800 to CE 1400 was moderately
warmer than multi-century periods both preceding and following it. This warmth, however, is dwarfed
by late 20th century warmth, which is observed to be unprecedented at least as far back as CE 200. The
coldest periods are the 6th, 15th, 17th, and 19th centuries.

The graph you created above was not shown in the original publication. The graph shown in their
original publication was smoother, because they filtered the data to remove some of the year-to-year
fluctuation. Use the moving-average technique to smooth the data yourself.

Overlay the original timeseries and the filtered timeseries. Comment on what you see (you can go the extra step of
making adjustments to the filtered timeseries so that x and y axis values are correctly aligned, given that the filter we've talked about can't handle the very first and last data points).